In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import math,sys

In [ ]:
##### Watts-Strogatz model parameters #####

# Number of nodes (graph vertices)
n=200
# Degree of the initial ring-lattice
k=10
# Resolution (number of different values of p, the rewiring probability)
r=20

# Generate a set of r logarithmically spaced values of p
p_range=[0] + np.logspace( -4, 0, r ).tolist()

###### For each p, compute the characterstic measures of the Watt-Strogatz model (n,k,p) ####
clustering = []
sp_length  = []
cc_rg = []
sp_rg = []

for p in p_range:
    # repeat the calculation 10 times and find the average value
    sp = 0
    cc = 0
    for i in range(10):
        # Create a Watts-Strogatz graph
        graph = nx.watts_strogatz_graph( n, k, p )
        
        # Compute the average shortest-path length L(p)
        sp+= nx.average_shortest_path_length( graph ) 

        # Compute the clustering coefficient C(p)
        cc+=nx.average_clustering( graph ) 
    sp_length.append(sp/10)
    clustering.append(cc/10)
    
    print "[p=%.4f]: Clustering coefficient C(p)=%.4f,  Avg shortest path L(p)=%.4f" % (p, clustering[-1], sp_length[-1]) 
    sys.stdout.flush()

##### Plot L(p)/L(0) and C(p)/C(0) #####
plt.figure(1,figsize=(9,9))

plt.semilogx(p_range[1:] , [L / sp_length[0]  for L in sp_length[1:]]  , '--sb', markersize=10, label='L(p)/L(0) - avg. shortest path')
plt.semilogx(p_range[1:] , [C / clustering[0] for C in clustering[1:]] , 'or', markersize=10, label='C(p)/C(0) - clustering coefficient')
plt.semilogx(p_range[1:] , [(1-p)**3.0 for p in p_range[1:]] , '--', color=[0.2,0.2,0.2], linewidth=3, markersize=12, label='C(p)/C(0) - Analytical clustering')

# Extend the plot region to see the points at both ends
lims = plt.xlim()
plt.xlim([lims[0]*0.9, lims[1]*1.1]) 
lims = plt.ylim()
plt.ylim([lims[0]*0.95, lims[1]*1.05]) 


plt.title( "Characteristics of Watts-Strogatz model with n=%d and k=%d"%(n,k) , fontsize=16)
plt.xlabel( "Rewiring probability (p)" , fontsize=14)
plt.ylabel( "L(p)/L(0) and C(p)/C(0)" , fontsize=14)
plt.legend( loc='best' )
plt.show()

In [ ]:
graph = nx.read_pajek('/Users/xyzjayne/Documents/CP290E/Lecture Files/Lecture9Materials/comm32.net').to_undirected()
def fromMulti_to_Single(G):
    H = nx.Graph()
    for u,v,d in G.edges(data=True):
        H.add_edge(u,v)
    return H
graph = fromMulti_to_Single(graph)

In [ ]:
##### Determine whether a network can be a random graph or a small world
def random_graph_test(G):
    print 'Hypothesis: random graph model'
    G = max(nx.connected_component_subgraphs(G),key = len)
    N = len(G.nodes)
    print 'Number of nodes in largest connected components: {}'.format(N)
    
    k_avg = np.mean(dict(G.degree()).values())
    cc = nx.average_clustering(G)
    sp = nx.average_shortest_path_length(G)
    
    p_RG = k_avg / N
    sp_RG = np.log(N)/np.log(k_avg)
    print 'Clustering coefficient \n Theoretical: {}, actual: {}'.format(p_RG,cc)
    print 'Difference: {}%'.format(int((cc-p_RG)/p_RG*100))
    print 'Average shortest path length \n Theoretical: {}, actual: {}'.format(sp_RG,sp)
    print 'Difference: {}%'.format(int((sp-sp_RG)/sp_RG*100))
    
random_graph_test(graph)

In [ ]:
def small_world_test(G):
    print 'Hypothesis: small world model'
    G = max(nx.connected_component_subgraphs(G),key = len)
    N = len(G.nodes)
    print 'Number of nodes in largest connected components: {}'.format(N)
    
    k_avg = np.mean(dict(G.degree()).values())
    cc = nx.average_clustering(G)
    sp = nx.average_shortest_path_length(G)
    
    SW_0 = nx.watts_strogatz_graph( N, int(k_avg), 0 )
    C_0 = nx.average_clustering(SW_0)
    
    # find p given clustering coefficient
    p_SW = 1 - (cc/C_0)**(1./3)

    print ('Value of p that produces the right clustering coefficient \n {}'.format(p_SW))
    print ('Within typical small world p value 0.01 - 0.1? \n {}'.format(p_SW>0.01 and p_SW<0.1))
    
small_world_test(graph)